In [85]:
# Load in the python script containing the same code as the load the data notebook
%run loadData.py
# now we can access train, dev, and test
# along with trainSents, devSents testSents

In [86]:
documents = testSents[0]
questions = test[0]

## Joshi part

In [87]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [88]:
# Tuning functions

import nltk
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

# Follow lemmatize function from guide notebook: WSTA_N1B_preprocessing.ipynb
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma

word_tokenizer = nltk.tokenize.WordPunctTokenizer() #word_tokenize #tokenize.regexp.WordPunctTokenizer()

def pre_process(line):
    tokenized_sentence = word_tokenizer.tokenize(line.lower())
    lemmatized_sentence = [lemmatize(token) for token in tokenized_sentence]
    return lemmatized_sentence

In [89]:
# Core functions

def vectorize_documents(text_documents):

    vectorizer = TfidfVectorizer(stop_words='english', tokenizer=pre_process)
    vector_documents = vectorizer.fit_transform(text_documents)
    
    return [vector_documents, vectorizer]

def vectorize_query(vectorizer, text_query):
    return vectorizer.transform([text_query])

def process_neighbours(vector_documents):
    
    neighbours = NearestNeighbors(1, algorithm="brute", metric="cosine")
    neighbours.fit(vector_documents)
    
    return neighbours

def closest_document(neighbours, vector_query):

    result = neighbours.kneighbors(vector_query, 1, return_distance=True)

    result_index = result[1][0][0]
    result_distance = result[0][0][0]
    
    return [result_distance, result_index]

In [90]:
def generatePartAOutput(qs, sents):
    # Output for part A
    partAOutput = []
    for i in range (0, len(qs)):
        documents = sents[i]
        questions = qs[i]

        vector_documents, vectorizer = vectorize_documents(documents)
        analyze = vectorizer.build_analyzer()
        neighbours = process_neighbours(vector_documents)

        for j in range (0, len(questions)):
            text_query = questions[j]["question"]
            vector_query = vectorize_query(vectorizer, text_query)
            result_similarity, result_index  = closest_document(neighbours, vector_query)
            partAOutput.append((i,j,result_index))
    return partAOutput

partADevAnswers = generatePartAOutput(dev, devSents)
partATestAnswers = generatePartAOutput(test, testSents)

## Alex part

In [8]:
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

# The required jar files : https://nlp.stanford.edu/software/CRF-NER.shtml#Download
# It's 171mb so I've added to the gitignore
# If you download it, and rename the folder name "stanford" in the main directory
classifier = './stanford/classifiers/english.all.3class.distsim.crf.ser.gz'
jar = './stanford/stanford-ner.jar'

sTagger = StanfordNERTagger(classifier,jar)

In [9]:
import pickle # Useful for read / write of list file
import os #Needed to check if file exists

In [10]:
# Lets store the stanford tagger output in a file
# This function returns the tagging output of stanford for each dataset
# with datasetName - 'train', 'dev', test' 

def getStanfordTagging(datasetName):
    fnameTrain = './preCompTags/stanfordTaggedTrain.txt'
    fnameDev = './preCompTags/stanfordTaggedDev.txt'
    fnameTest = './preCompTags/stanfordTaggedTest.txt'
    
    theFilePath = ''
    theSents = []
    if (datasetName == 'train'):
        theFilePath = fnameTrain
        theSents = trainSents
    elif (datasetName == 'dev'):
        theFilePath = fnameDev
        theSents = devSents
    elif (datasetName == 'test'):
        theFilePath = fnameTest
        theSents = testSents
    else :
        raise ValueError('Incorrect datasetName: ' + datasetName + ', choose from - "train", "dev", "test" ') 
    if (os.path.exists(theFilePath)):
        with open(theFilePath, "rb") as fp:
            stanfordTags = pickle.load(fp)
            return stanfordTags
    
    else :
        #Need to create taggings!
        taggedSentsList = []
        for sents in theSents:
            tokenisedSents = [word_tokenize(sent) for sent in sents]
            classifiedSents = sTagger.tag_sents(tokenisedSents)
            taggedSentsList.append(classifiedSents)
        #And save them
        with open(theFilePath, "wb") as fp: 
            pickle.dump(taggedSentsList, fp)
        return taggedSentsList
    

In [11]:
taggedTrain = getStanfordTagging('train')
taggedDev = getStanfordTagging('dev')
taggedTest = getStanfordTagging('test')

In [12]:
import re
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))


# Given a stanford tagged list, refines the list by:\n",
# Grouping all contiguous words with the same tag\n",
# Relabels Organisations as Other\n",
# Labels Number\n",
def refineWordTags(taggedWordList):
    newWordTags = []
    for (word, tag) in taggedWordList:
        if (tag == 'ORGANIZATION'):
            tag = 'O'
        if (tag == 'O'):
            #Might be a number
            if isNumber(word):
                tag = 'NUMBER'
            elif isCapitalised(word):
                tag = 'OTHERCAP'
            elif word in preUnits:
                tag = 'PRENUM'
            elif word in postUnits:
                tag = 'POSTNUM'
            elif isStopWord(word):
                tag = 'STOPWORD'
            elif isPunctuation(word):
                tag = 'PUNC'

        newWordTags.append((word, tag))
    
    newWordTags = combineTags (newWordTags)
    return newWordTags
        
def combineTags(wordTags):
    newTags = []
    prevWord = wordTags[0][0]
    prevTag = wordTags[0][1]
    for (word, tag) in wordTags[1:]:
        if tag == 'NUMBER' and prevTag == 'PRENUM':
            prevTag = 'NUMBER'
        elif prevTag == 'PRENUM':
            prevTag = 'O'
        if tag == 'POSTNUM' and prevTag == "NUMBER":
            tag = "NUMBER"
        elif tag == "POSTNUM":
            tag = "O"
        newTags.append((prevWord, prevTag))
        prevWord = word
        prevTag = tag
    newTags.append((prevWord, prevTag))
    
    newNewTags = []
    prevWord = newTags[0][0]
    prevTag = newTags[0][1]
    if (prevTag == "OTHERCAP"):
        prevTag = "O"
    for (word, tag) in wordTags[1:]:            
        if tag == prevTag :
            prevWord += ' ' + word
        else :
            newNewTags.append((prevWord, prevTag))
            prevWord = word
            prevTag = tag
    newNewTags.append((prevWord, prevTag))
    return newNewTags

# Thanks for this list to save me typing it : http://stackoverflow.com/questions/493174/is-there-a-way-to-convert-number-words-to-integers\n",
numInWords = ["zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
        "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
        "sixteen", "seventeen", "eighteen", "nineteen", "twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety"
       , "hundred", "thousand", "million", "billion", "trillion"]

punctuation = ['.',',',';',':']

def isPunctuation(word):
    return word in punctuation
def isCapitalised (word):
    if len(word) == 0:
        return False
    return word[0].isupper()

# Obtained from training data
postUnits = [u'%', u'century', u'years', u'percent', u'years ago', u'days', u'months', u'km', u'hours', u'times', u'inches', u'\xb0C', u'minutes', u'acres', u'\xb0F', u'weeks', u'people', u'sq mi', u'mi', u'ft', u'feet', u'metres', u'mm', u'square miles', u'miles', u'pm', u'per cent', u'year', u'copies', u'yuan', u'men', u'square feet', u'third', u'kilometres', u'nm', u'tonnes', u'species', u'decades', u'barrels', u'tons', u'largest', u'centuries', u'km2']
preUnits = [u'$', u'around', u'late', u'early', u'nearly', u'since', u'approximately', u'number']

# Returns true if the word represents a number\n",
def isNumber(word):
    pattern = ".?(\\d)+((,|.)(\\d)+)*"
    if re.match(pattern,word) :
        return True
    if word.lower() in numInWords:
        return True
    return False

def isStopWord(word):
    return word.lower() in stop_words


In [13]:
# Get the output for part B:
# In the form [(docID, questID, entities)]
def generatePartBOutput(qs, partAOutput, taggedSents):   
    partBOutput = []
    for (docIndex, questionIndex, sentenceIndex) in partAOutput:
        stanfordTags = taggedSents[docIndex][sentenceIndex]
        newTags = refineWordTags(stanfordTags)
        question = qs[docIndex][questionIndex]["question"]
        partBOutput.append((docIndex, questionIndex, sentenceIndex, newTags))
    return partBOutput

partBDevAnswers = generatePartBOutput(dev, partADevAnswers, taggedDev)
partBTestAnswers = generatePartBOutput(test, partATestAnswers, taggedTest)

In [14]:
partBDevAnswers[0]

(0,
 0,
 1,
 [(u'Night-vision devices using active near-infrared illumination allow', 'O'),
  (u'people', 'POSTNUM'),
  (u'or', 'STOPWORD'),
  (u'animals', u'O'),
  (u'to be', 'STOPWORD'),
  (u'observed without', u'O'),
  (u'the', 'STOPWORD'),
  (u'observer', u'O'),
  (u'being', 'STOPWORD'),
  (u'detected', u'O'),
  (u'.', 'PUNC')])

In [ ]:
## Example output:
print "\nDocIndex: " + partBTestAnswers[0][0]
print "\nQuestionIndex: " + partBTestAnswers[0][1]
print 

print "\nPossibleAnswers: " 
print partBTestAnswers[0][3]

#Okay so this one is wrong, but hey! 

## Quick evaluation of the first two parts on the dev set:


In [103]:
def evaluateNERonDev():
    correct = []
    wrong = []
    for (docIndex, questionIndex, sentenceIndex, newTags) in partBDevAnswers:
        answer = dev[docIndex][questionIndex]["answer"]
        possAnswers = newTags 
        inThere = False
        for possAnswer in newTags:
            if possAnswer[0] == answer:
                inThere = True
                break
        if inThere:
            correct.append((docIndex, questionIndex, sentenceIndex, newTags))
        else :
            wrong.append((docIndex, questionIndex, sentenceIndex, newTags))
    return (correct, wrong)

In [100]:
(nerDevCorrect, nerDevWrong) = evaluateNERonDev()
print len(nerDevCorrect)
print len(nerDevWrong)

2409
6054


In [104]:
# For each question, evaluate if the answer is present as an entity

def evaluateAnswerRanking(questionsList,documentsList, numToEval):
    correct = []
    wrong = []
    #(corNER, wrongNER) = evaluateNER(questionsList,documentsList, numToEval)
    (nerDevCorrect, nerDevWrong) = evaluateNERonDev()
    entityListsWithAnswer = nerDevCorrect
    for (i,j,possAnswers) in entityListsWithAnswer:
        question = questionsList[i][j]["question"]
        answer =  questionsList[i][j]["answer"]
        print question
        print possAnswers
        #print answer
        answerPredicited = third_filter(question,second_filter(question,first_filter(question, possAnswers)),second_filter(question,first_filter(question, possAnswers))['answer_entities_list']) 
        #print answerPredicited
        #print '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%'
        # TODO add Dereks part here
        if (answerPredicited == answer):
            correct.append((i,j))
        else :
            wrong.append((i,j,answerPredicited))
        #print correct
    return (correct, wrong)
    

In [105]:
(corAns, wrongAns) = evaluateAnswerRanking(dev, devSents,len(dev))
print("Number Correct : " + str(len(corAns)))
print("Number incorrect: " + str(len(wrongAns)))
print ("Average correct : " + str((len(corAns) + 0.0) / (len(corAns)+len(wrongAns))))

ValueError: too many values to unpack

## Derek part

In [70]:
def detect_openclass_word(sameword_list):
    tagged_text = nltk.pos_tag(sameword_list)
    is_noun = lambda pos: pos[:2] == 'NN'
    is_verb = lambda pos: pos[:2] == 'VB'
    is_adjective = lambda pos: pos[:2] == 'JJ'
    is_adverb = lambda pos: pos[:2] == 'RB'
    
    nouns = [word for (word, pos) in tagged_text if is_noun(pos)] 
    verbs = [word for (word, pos) in tagged_text if is_verb(pos)] 
    adjectives = [word for (word, pos) in tagged_text if is_adjective(pos)] 
    adverbs = [word for (word, pos) in tagged_text if is_adverb(pos)] 
    return nouns+adjectives+adverbs

In [20]:
# Given a question, returns a tag for the answer form
# From PERSON, LOCATION, NUMBER, OTHER 
# Assuming question is lowercased
def getQuestionType(question):
    if 'Who' in question:
        return "PERSON"
    if 'where' in question:
        return "LOCATION"
    if 'How many' in question:
        return "NUMBER"
    if 'How much' in question:
        return "NUMBER"
    if 'When' in question:
        return "NUMBER"
    if 'what year' in question:
        return "NUMBER"
    if 'What year' in question:
        return "NUMBER"
    else:
        return "O"

In [21]:
#remove the non-word entities from answer-entities
def process_answer_entities(anwser_entities):
    new_anwser_entities =[]
    for (entity,entity_type) in anwser_entities:
        tokenized_entity = word_tokenize(entity)
        #print tokenized_entity
        temp = []
        for ele in tokenized_entity:
            if ele not in non_words:
                temp.append(ele)
        temp = ' '.join(temp)
        new_anwser_entities.append((temp,entity_type))
    return new_anwser_entities      


In [78]:

#first:  whose anwsers all appear in the questions rank the lowest
#assumption: input question in its dictionary value
from string import punctuation  
non_words = list(punctuation)

def first_filter(question, anwser_entities):
    ranking_dict_1 = {}
    ranking_list = []
    merge_list = []
    answer_entities_list = []
    anwser_entities = process_answer_entities(anwser_entities)
    
    for entity in anwser_entities:
        answer_entities_list.append(entity[0])
        if entity[0] in question:
            #print entity[0]
            #if entity[0].lower() not in stop_words and entity[0]!='' and entity[0].lower() not in non_words:
            if entity[1] != 'STOPWORD' and entity[1] !='PUNC':   
                #print entity[0]
                merge_list.append(entity[0])
        else:
            #if entity[0].lower() not in stop_words and entity[0]!='' and entity[0].lower() not in non_words:
            if entity[1] != 'STOPWORD' and entity[1] !='PUNC':  
                ranking_list.append(entity)
                
    #get rid of the not noun phrases   
    #print merge_list
    
    final_merge_list = []
    for phrase in merge_list:
        tokenized_phrase = word_tokenize(phrase)
        for word in tokenized_phrase:
            if word not in stop_words and word.lower() not in non_words:
                #print phrase
                final_merge_list.append(phrase)
                
    final_ranking_list = []            
    for entity in ranking_list:
        if entity[1] != 'STOPWORD':
            final_ranking_list.append(entity)
    
    #detect the open-class word in here for easier process            
    final_merge_list = set(detect_openclass_word(merge_list))
    ranking_dict_1["ranking_list"] = final_ranking_list
    ranking_dict_1["same_word_list"] = final_merge_list
    ranking_dict_1["answer_entities_list"] = answer_entities_list
    return ranking_dict_1


In [23]:

#second: answers which match the question type should be ranked higher than those that dont

#assumption: save questions' type in the dictionary format quesiton1 = 
#{u'answer': u'long playing',u'answer_sentence': 2, u'question':......., 'question_type:'PERSON'}

def second_filter(question, ranking_dict_1):
    question_with_type ={}
    question_with_type['question_type']= getQuestionType(question)
    question_with_type['question'] = question
    #print question_with_type
    ranking_dict_2 = {}
    ranking_list =[]
    merge_list = []
    for entity in ranking_dict_1["ranking_list"]:
        if question_with_type['question_type'] == 'O':
            ranking_list.append(entity[0])
        else:
            if entity[1] == question_with_type['question_type']:
                ranking_list.append(entity[0])
            else:
                merge_list.append(entity[0])
    ranking_dict_2["same_word_list"] = ranking_dict_1["same_word_list"]
    ranking_dict_2["ranking_list"] = ranking_list
    ranking_dict_2["Other_tags_list"] = merge_list
    ranking_dict_2["answer_entities_list"] = ranking_dict_1["answer_entities_list"]
    return ranking_dict_2
    

In [24]:

#Thrid: based on second, the prefered entity should be the one which is close in 
#the sentence to a closed-class word form the question
from collections import OrderedDict

def cal_distance_words(entity,same_words, anwser_entities):
    temp = 0
    for same_word in same_words:
        temp += abs(anwser_entities.index(entity) - anwser_entities.index(same_word))
    return float(temp)/float(len(same_words))

def sort_orderedDict(orderdict):
    return OrderedDict(sorted(orderdict.items(), key = lambda x:x[1], reverse = False))
        

def third_filter(question,second_filter,anwser_entities):
    dict_ranking ={}
    if (len(second_filter["same_word_list"])==0):
        if len(second_filter['ranking_list']) != 0:
            return second_filter['ranking_list'][0]
        else:
            return second_filter["Other_tags_list"][0]
            #for entity in second_filter["Other_tags_list"]:
                #dict_ranking[entity]= cal_distance_words(entity, second_filter["same_word_list"],anwser_entities)
            #dict_ranking = sort_orderedDict(dict_ranking)
            #if len(dict_ranking.items()) ==0:
                #return 0
            #else:
            #return dict_ranking.items()[0][0]
    else:
        #if len(second_filter['ranking_list']) != 0:
        for entity in second_filter["ranking_list"]:
            dict_ranking[entity]= cal_distance_words(entity, second_filter["same_word_list"],anwser_entities)
        #else:
            #if len(second_filter["Other_tags_list"]) !=0:
                #for entity in second_filter["Other_tags_list"]:
                    #dict_ranking[entity]= cal_distance_words(entity, second_filter["same_word_list"],anwser_entities)
            #print dict_ranking
            #else:
                #return 0
        dict_ranking = sort_orderedDict(dict_ranking)
            #print dict_ranking
        if len(dict_ranking.items()) ==0:
            return 0
        else:
            return dict_ranking.items()[0][0]


In [ ]:
Along with industrial and medical, in what applications is infrared radiation used?
[(u'Infrared radiation', 'O'), (u'is', 'STOPWORD'), (u'used', u'O'), (u'in', 'STOPWORD'), (u'industrial', u'O'), (u',', 'PUNC'), (u'scientific', u'O'), (u',', 'PUNC'), (u'and', 'STOPWORD'), (u'medical applications', u'O'), (u'.', 'PUNC')]
scientific
Infrared radiation

In [66]:
question = "Along with industrial and medical, in what applications is infrared radiation used?"
a = [(u'Infrared radiation', 'O'), (u'is', 'STOPWORD'), (u'used', u'O'), (u'in', 'STOPWORD'), (u'industrial', u'O'), (u',', 'PUNC'), (u'scientific', u'O'), (u',', 'PUNC'), (u'and', 'STOPWORD'), (u'medical applications', u'O'), (u'.', 'PUNC')]


In [71]:
test = first_filter(question,a)
print test

used
industrial
{'ranking_list': [(u'Infrared radiation', 'O'), (u'scientific', u'O'), (u'medical applications', u'O')], 'answer_entities_list': [u'Infrared radiation', u'is', u'used', u'in', u'industrial', '', u'scientific', '', u'and', u'medical applications', ''], 'same_word_list': set([u'industrial'])}


In [72]:
test2 = second_filter(question, test)
print test2

{'ranking_list': [u'Infrared radiation', u'scientific', u'medical applications'], 'Other_tags_list': [], 'answer_entities_list': [u'Infrared radiation', u'is', u'used', u'in', u'industrial', '', u'scientific', '', u'and', u'medical applications', ''], 'same_word_list': set([u'industrial'])}


In [73]:
#print cal_distance_words('Melbourne',test2["same_word_list"],test2["answer_entities_list"])
test3 = third_filter(question, test2,test2["answer_entities_list"])
print test3

scientific


## Evaluation

In [106]:
# For each question, evaluate if the answer is present as an entity

def evaluateAnswerRanking(questionsList,documentsList, numToEval):
    correct = []
    wrong = []
    (nerDevCorrect, nerDevWrong) = evaluateNERonDev()
    #(corNER, wrongNER) = evaluateNER()
    entityListsWithAnswer = nerDevCorrect
    for (i,j,something_not_sure,possAnswers) in entityListsWithAnswer:
        question = questionsList[i][j]["question"]
        answer =  questionsList[i][j]["answer"]
        #print question
        #print possAnswers
        #print answer
        answerPredicited = third_filter(question,second_filter(question,first_filter(question, possAnswers)),second_filter(question,first_filter(question, possAnswers))['answer_entities_list']) 
        #print answerPredicited
        #print '%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%'
        # TODO add Dereks part here
        if (answerPredicited == answer):
            correct.append((i,j))
        else :
            wrong.append((i,j,answerPredicited))
        #print correct
    return (correct, wrong)
    

In [107]:
(corAns, wrongAns) = evaluateAnswerRanking(dev, devSents,len(dev))
print("Number Correct : " + str(len(corAns)))
print("Number incorrect: " + str(len(wrongAns)))
print ("Average correct : " + str((len(corAns) + 0.0) / (len(corAns)+len(wrongAns))))

IndexError: string index out of range

## Derek submission

In [ ]:
answer_list = []
for ele in partBOutput:
    answer_dict = {}
    question = ele[0]
    possAnswers = ele[1]
    answer = third_filter(question,second_filter(question,first_filter(question, possAnswers)),second_filter(question,first_filter(question, possAnswers))['answer_entities_list'])
    answer_dict['sentence'] = possAnswers
    answer_dict['question'] = question
    answer_dict['answer'] = answer
    answer_list.append(answer_dict)

In [ ]:
import csv
headers = ['id', 'answer']

with open('submit.csv','w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    for index in range(len(answer_list)):
        if isinstance( answer_list[index]['answer'], int):
            f_csv.writerows([{'id':index+1,'answer':answer_list[index]['answer']}])
        else:
            f_csv.writerows([{'id':index+1,'answer':answer_list[index]['answer'].encode("utf-8")}])